In [35]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set api token 
QX_TOKEN = "96c86855a5c381875a4ea3a956a7deafbbbbb82e5912ec73655c7c444d2597a7364eda2fa558e4e37f672640029443216899825f945d5a0684e31b796837ce61"
QX_URL = "https://quantumexperience.ng.bluemix.net/api"


#register token and url
register(QX_TOKEN, QX_URL)

backend = 'local_qasm_simulator'

# prepare a state of all zeros
def prepareZeros(qc):
    qc.h(qr[1])
    qc.cx(qr[1],qr[0])
    qc.cx(qr[1],qr[2])
    qc.cx(qr[2],qr[3])

    
# compile into different states
def compilation(qc,i,perm):
        if i == 0: 
            return
        elif i == 1:
            qc.x(qr[0])
            qc.x(qr[1])
        elif i == 2:
            qc.x(qr[0])
            qc.x(qr[2])
        elif i == 3:
            qc.x(qr[2])
            qc.x(qr[1])
            
# add phase gate
def addPhase(qc):
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    qc.h(qr[3])
    qc.cx(qr[1],qr[0])
    qc.s(qr[0])
    qc.cx(qr[1],qr[0])
    
# remove phase gate
def removePhase(qc,perm):
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.sdg(qr[perm[0]])
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.h(qr[3])
    qc.h(qr[2])
    qc.h(qr[1])
    qc.h(qr[0])
    
# parse a string of gens
def addCliffGens(qc, string, perm):
    for char in string:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.s(qr[0])
            qc.s(qr[1])
            qc.s(qr[2])
            qc.s(qr[3])
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)        

# find string for generators
def findGenString(mat):
    for key, value in signedGroup.items():
        if np.array_equal(value, mat):
            return key
    print("OH NO")
    return None
        
# invert cliff gens 
def invertCliffGens(qc, mat, perm):
    genString = findGenString(mat)
    for char in genString[::-1]:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.sdg(qr[0])
            qc.sdg(qr[1])
            qc.sdg(qr[2])
            qc.sdg(qr[3])
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)

# adds random real clifford
def addRandCliff(qc, mat,perm):
    genString = random.choice(list(r2group.keys()))
    addCliffGens(qc,genString,perm)
    qc.barrier(qr)
    return np.matmul(mat,r2group[genString])

pickle_in = open("r2group.pkl", "rb")
r2group = pickle.load(pickle_in)
pickle_in2 = open("r2group.pickle", "rb")
signedGroup = pickle.load(pickle_in2)

perm = {0: 0, 1: 1, 2: 2, 3: 3}

# used for reindexing qubits
def swap(perm, a, b):
    temp = perm[a]
    perm[a] = perm[b]
    perm[b] = temp
    
# converts 0-3 to binary strings
def toBinString(num):
    if num ==0:
        return "00"
    elif num == 1:
        return "01"
    elif num == 2:
        return "10"
    elif num == 3:
        return "11"
    else:
        print("OH NO")


trials = 1
reps = 9
initialStates = 4
shots = 1024

startTime = time.time()

for k in range(initialStates):

    for i in range(trials):
        length = 2+3*i
        print(length)
        counts = {}

        # run our trials
        for i in range(reps):

            # run one trial of given length 
            mat = r2group[""]
            n = 4
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr, cr)
            
            # prepare encoded zeros
            prepareZeros(qc) 
            
            # optional: add phase
            addPhase(qc)

            # add the first m-1 clifford gates
            for i in range(length):
                mat = addRandCliff(qc,mat,perm)

            # invert everything
            invertCliffGens(qc,mat, perm)
            
            # optional: remove phase
            removePhase(qc, perm)

            # get the qc compiled into one of the 4 different states
            compilation(qc,k, perm)
            
            # measure
            for i in range(n):
                qc.measure(qr[perm[i]], cr[i])      
            
            print("got")
            job = execute(qc, backend, shots = shots)
            print("here")
            result = job.result()
            print("nice")
            trial = result.get_counts(qc)
            print(trial)
            
            # clean the data
            data = {}
            for key in trial:
                ohoh = ['0000', '1111']
                ohone = ['1100', '0011']
                oneoh = ['1010', '0101']
                oneone = ['0110', '1001']
                if key not in ohoh+ohone+oneoh+oneone:
                    continue
                if key in ohoh:
                    if "00" not in data.keys():
                        data["00"] = 0
                    data["00"] += trial[key]
                elif key in ohone:
                    if "01" not in data.keys():
                        data["01"] = 0
                    data["01"] += trial[key]
                elif key in oneoh:
                    if "10" not in data.keys():
                        data["10"] = 0
                    data["10"] += trial[key]
                elif key in oneone:
                    if "11" not in data.keys():
                        data["11"] = 0
                    data["11"] += trial[key]
            #plot_histogram(data)
            
            # optional: get success rate
            desired = toBinString(k)
            print(data[desired]/1024)
endTime = time.time()
print("Time elapsed: ", endTime-startTime)



  

Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\quantum_rb\Qconfig.py.
2
got
here
nice
{'0000': 522, '1111': 502}
1.0
got
here
nice
{'0000': 503, '1111': 521}
1.0
got
here
nice
{'0000': 512, '1111': 512}
1.0
got
here
nice
{'0000': 487, '1111': 537}
1.0
got
here
nice
{'0000': 527, '1111': 497}
1.0
got
here
nice
{'0000': 531, '1111': 493}
1.0
got
here
nice
{'0000': 526, '1111': 498}
1.0
got
here
nice
{'0000': 465, '1111': 559}
1.0
got
here
nice
{'0000': 488, '1111': 536}
1.0
2
got
here
nice
{'0011': 526, '1100': 498}
1.0
got
here
nice
{'0011': 512, '1100': 512}
1.0
got
here
nice
{'0011': 522, '1100': 502}
1.0
got
here
nice
{'0011': 539, '1100': 485}
1.0
got
here
nice
{'0011': 529, '1100': 495}
1.0
got
here
nice
{'0011': 503, '1100': 521}
1.0
got
here
nice
{'0011': 494, '1100': 530}
1.0
got
here
nice
{'0011': 494, '1100': 530}
1.0
got
here
nice
{'0011': 536, '1100': 488}
1.0
2
got
here
nice
{'0101': 523, '1010': 501}
1.0
got
here
nice
{'0101': 530, '1010': 494}
1.0
got
here
nice
{'010